<a href="https://colab.research.google.com/github/tinkvu/TravelRecommenderUsingLLM/blob/main/Notebooks/Dublin_Tourist_%2B_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/tinkvu/TravelRecommenderUsingLLM/
!ls

Cloning into 'TravelRecommenderUsingLLM'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 95 (delta 30), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (95/95), 1.81 MiB | 3.39 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Data  sample_data  TravelRecommenderUsingLLM


#Data Cleaning

In [13]:
#import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import string

from wordcloud import WordCloud
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [9]:
import pandas as pd

# Read the CSV file and drop the first column
df = pd.read_csv('/content/TravelRecommenderUsingLLM/Raw Data/Processed/processed_datas.csv').iloc[:, 1:]

# Print the DataFrame
df.head()

,place_name,rating,review
0,14 Henrietta Street,4.875,The museum is very good. I highly recommend it...
1,1916 Rebellion Walking Tour,4.975,"Lorcan is a professional guide, passionate abo..."
2,3fe Grand Canal Street,4.700,Delicious coffee and breakfast in Dublin. Lots...
3,Arthur Guinness Dwelling House,5.000,Who would think Guinness Beer at that time?
4,As One,4.450,Came here on a Saturday morning without a book...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   place_name  217 non-null    object 
 1   rating      217 non-null    float64
 2   review      216 non-null    object 
dtypes: float64(1), object(2)
memory usage: 5.2+ KB


In [15]:
# Downloading the NLTK stopwords dataset.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize NLTK's English stopwords and WordNet lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to clean and normalize text
def clean_text(text):
    if isinstance(text, str):  # Check if the input is a string
        text = text.lower()  # Lowercase
        text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        tokens = word_tokenize(text)  # Tokenize text into words
        tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Remove stopwords and lemmatize
        cleaned_text = ' '.join(tokens)  # Join tokens back into a single string
        return cleaned_text
    else:
        return ''  # Return empty string for non-string inputs (like NaN)

# Apply text cleaning
df['review1'] = df['review'].apply(clean_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
df.tail()

,place_name,rating,review,review1
212,Wellington Monument,4.625,Very impressive monument and great place to re...,impressive monument great place relax grass su...
213,White Rabbit Cafe,4.200,Theres this really cool little coffee shop on ...,there really cool little coffee shop busy stre...
214,Wolfe Tone Park,3.925,A popular little square with seating and grass...,popular little square seating grassed area the...
215,World Poverty Stone,4.450,Impressive lean face people figures made of st...,impressive lean face people figure made stone ...
216,"laine, my love",4.350,Nice use of a small space. Really affordable a...,nice use small space really affordable good po...


In [22]:
df=df.drop(['review','rating'],axis=1)

In [27]:
df['review']=df['review1']
df=df.drop(['review1'],axis=1)

In [28]:
df.head()

,place_name,review
0,14 Henrietta Street,museum good highly recommend know georgian hou...
1,1916 Rebellion Walking Tour,lorcan professional guide passionate irish his...
2,3fe Grand Canal Street,delicious coffee breakfast dublin lot indoor s...
3,Arthur Guinness Dwelling House,would think guinness beer time
4,As One,came saturday morning without booking got tabl...


In [29]:
df.to_csv('cleaned_data.csv',index=False)

#LLM

In [1]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
ERROR: pip's de

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

model_id = "meta-llama/Meta-Llama-3-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=os.environ['HF_TOKEN'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [9]:
text = "Where can i get the best beer in dublin"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
output = model.generate(**inputs, max_length=100, num_return_sequences=1)
#outputs = model.generate(**inputs, max_new_tokens=60)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Where can i get the best beer in dublin?
Dublin has a wealth of pubs, bars and beer houses, but which are the best? There’s a great variety of pubs, bars and beer houses in Dublin, and each one has its own unique charm. Whether you’re looking for a traditional Irish pub, a modern beer house, or a cosy local, Dublin has it all. Here’s a guide to the best beer in Dublin, with some of the most popular pubs, bars


In [ ]:
os.environ["WANDB_DISABLED"] = "true"
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
